<a href="https://colab.research.google.com/github/marcoss00/metaheuristicas_otimizacao_comb/blob/master/3_Trabalho_Aplica%C3%A7%C3%A3o_de_Metaheur%C3%ADstica_ao_Problema_da_Mochila.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import random
import math

# Define o problema da mochila
valores = [91, 20, 81, 50, 76, 30, 85, 65, 44, 70, 60, 25, 88, 55, 33, 95, 18,
           62, 42, 75, 51, 68, 39, 90, 58, 72, 29, 64, 82, 47, 78, 40, 71, 54,
           89, 26, 77, 63, 45, 84, 22, 69, 52, 80, 37, 61, 83, 48, 73, 57, 92,
           24, 67, 53, 86, 28, 66, 79, 49, 74, 41, 70, 59, 94, 19, 76, 46, 60,
           34, 75, 56, 17, 65, 87, 16, 68, 50, 81, 27, 62, 78, 43, 72, 61, 93,
           23, 71, 52, 85, 38, 67, 80, 36, 63, 77, 44, 70, 58, 88, 32]           # valor de cada item
pesos = [74, 16, 58, 35, 72, 15, 77, 52, 28, 63, 45, 19, 66, 40, 23, 81, 12, 50,
         31, 68, 37, 54, 25, 79, 43, 60, 21, 48, 70, 33, 64, 27, 56, 39, 75, 18,
         62, 47, 29, 76, 14, 53, 36, 67, 24, 49, 71, 32, 59, 42, 78, 17, 55, 38,
         73, 20, 51, 69, 34, 61, 26, 57, 44, 80, 13, 65, 30, 46, 22, 63, 41, 11,
         50, 77, 10, 54, 35, 72, 19, 48, 66, 31, 58, 45, 82, 15, 60, 37, 74, 28,
         52, 69, 25, 47, 64, 33, 59, 43, 76, 21]                                 # peso de cada item
capacidade_mochila = 1550                                                        # peso máximo que a mochila pode carregar
tamanho_populacao = 50                                                           # número de indivíduos na população
numero_geracoes = 399                                                            # número de gerações para evoluir
taxa_mutacao = 0.03                                                              # chance de mutação de um gene
numero_pais_selecionados = 2                                                     # número de pais para cruzamento

# Gera um indivíduo aleatório (uma possível solução)
def criar_individuo(tamanho_genoma):
    # Cada gene é 0 ou 1, indicando se o item está ou não na mochila
    return [random.randint(0, 1) for _ in range(tamanho_genoma)]

# Cria a população inicial
def criar_populacao(tamanho_populacao, tamanho_genoma):
    return [criar_individuo(tamanho_genoma) for _ in range(tamanho_populacao)]

# Avalia o valor total do indivíduo (solução)
def avaliar_individuo(individuo, pesos, valores, capacidade):
    peso_total = 0
    valor_total = 0
    for i in range(len(individuo)):
        if individuo[i] == 1:
            peso_total += pesos[i]
            valor_total += valores[i]
    # Se ultrapassar a capacidade, penaliza o valor
    if peso_total > capacidade:
        excesso = peso_total - capacidade
        penalizacao = excesso * 3  # penalização proporcional
        return max(0, valor_total - penalizacao)
    return valor_total

# Avalia toda a população
def avaliar_populacao(populacao, pesos, valores, capacidade):
    return [avaliar_individuo(ind, pesos, valores, capacidade) for ind in populacao]

# Seleciona os melhores indivíduos (elitismo simples)
def selecionar_pais(populacao, avaliacoes, numero_pais):
    # Combina indivíduos com seus valores
    combinados = list(zip(populacao, avaliacoes))
    # Ordena pelo valor (fitness), do melhor para o pior
    combinados.sort(key=lambda x: x[1], reverse=True)
    # Retorna os melhores indivíduos
    pais = [ind for ind, _ in combinados[:numero_pais]]
    return pais

# Realiza cruzamento (crossover) de dois pais para gerar dois filhos
def cruzar(pai1, pai2):
    ponto_corte = random.randint(1, len(pai1) - 1)
    filho1 = pai1[:ponto_corte] + pai2[ponto_corte:]
    filho2 = pai2[:ponto_corte] + pai1[ponto_corte:]
    return filho1, filho2

# Aplica mutação aleatória em um indivíduo
def mutar(individuo, taxa_mutacao):
    for i in range(len(individuo)):
        if random.random() < taxa_mutacao:
            individuo[i] = 1 - individuo[i]  # inverte o bit
    return individuo

# Gera uma nova população a partir dos pais
def gerar_nova_populacao(pais, tamanho_populacao, taxa_mutacao):
    nova_populacao = []
    while len(nova_populacao) < tamanho_populacao:
        pai1 = random.choice(pais)
        pai2 = random.choice(pais)
        filho1, filho2 = cruzar(pai1, pai2)
        filho1 = mutar(filho1, taxa_mutacao)
        filho2 = mutar(filho2, taxa_mutacao)
        nova_populacao.append(filho1)
        if len(nova_populacao) < tamanho_populacao:
            nova_populacao.append(filho2)
    return nova_populacao

def estatisticas_da_geracao(avaliacoes):
    n = len(avaliacoes)
    media = sum(avaliacoes) / n

    # Cálculo do desvio padrão
    variancia = sum((x - media) ** 2 for x in avaliacoes) / n
    desvio_padrao = math.sqrt(variancia)

    return media, desvio_padrao

melhor_individuo_global = []
melhor_valor_global = []
pior_valor_global = []

def algoritmo_genetico():
    tamanho_genoma = len(valores)
    populacao = criar_populacao(tamanho_populacao, tamanho_genoma)

    for geracao in range(numero_geracoes):
        avaliacoes = avaliar_populacao(populacao, pesos, valores, capacidade_mochila)
        melhor_valor = max(avaliacoes)
        pior_valor = min(avaliacoes)
        melhor_individuo = populacao[avaliacoes.index(melhor_valor)]
        pior_individuo = populacao[avaliacoes.index(pior_valor)]
        # print(f"Geração {geracao+1}: Melhor valor = {melhor_valor}, Solução = {melhor_individuo}")

        media, desvio = estatisticas_da_geracao(avaliacoes)

        pais = selecionar_pais(populacao, avaliacoes, numero_pais_selecionados)
        populacao = gerar_nova_populacao(pais, tamanho_populacao, taxa_mutacao)

    melhor_individuo_global.append(melhor_individuo)
    melhor_valor_global.append(melhor_valor)
    pior_valor_global.append(pior_valor)
    print("Solução encontrada:")
    print(f"Melhor individuo: {melhor_individuo}")
    print(f"Valor do melhor individuo: {melhor_valor}")
    print(f"Valor do pior individuo: {pior_valor}")
    print(f"Média: {media}")
    print(f"Desvio padrão: {desvio}")

for _ in range(20):
  print(f"\n\nExecução {_ + 1}")
  algoritmo_genetico()


avaliacoes_melhores = avaliar_populacao(melhor_individuo_global, pesos, valores, capacidade_mochila)
media_melhores_global, desvio_melhores_global = estatisticas_da_geracao(avaliacoes_melhores)
print("\n\nSolução encontrada:")
print(f"Melhor individuo global: {melhor_individuo_global[avaliacoes_melhores.index(max(avaliacoes_melhores))]}")
print(f"Valor do melhor individuo global: {max(melhor_valor_global)}")
print(f"Valor do pior individuo global: {min(pior_valor_global)}")
print(f"Média global dos melhores: {media_melhores_global}")
print(f"Desvio padrão global dos melhores: {desvio_melhores_global}")



Execução 1
Solução encontrada:
Melhor individuo: [0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1]
Valor do melhor individuo: 2107
Valor do pior individuo: 1715
Média: 1965.44
Desvio padrão: 98.9135299137585


Execução 2
Solução encontrada:
Melhor individuo: [0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1]
Valor do melhor individuo: 2121
Valor do pior individuo: 1429
Média: 1949.98
Desvio padrão: 139.16170306517523


Execução 3
Solução encontrada:
Melhor individuo: [0, 1, 1, 0, 0, 1, 0, 0, 